## Assignment-07 First Step of using machine learning and models.

In [1]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyltp import Segmentor
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import scipy
%matplotlib inline

### Step1: 数据分析

请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [2]:
fname = "D:\\Github\\NLP\\Artificial_Intelligence_for_NLP\\Week_05_0803_pyltp\\sqlResult_1558435.csv"

In [3]:
data = pd.read_csv(fname, encoding='gb18030')

In [190]:
data.head(1)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm


In [5]:
xinhua_news = data[data.source == '新华社']

In [6]:
len(xinhua_news) / len(data)

0.8778051801676133

### 任务描述

报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1566105348906&di=ee9a2de91207767364853d4decc6cca3&imgtype=0&src=http%3A%2F%2Fmmbiz.qpic.cn%2Fmmbiz_png%2FTicO2kbP6Ao5sCsSQDpehZiczLdC6hDCNvoicjcOCEKX2bLxBc9gVOw28zHyFibfIWq9ceRibP6HDTKReGkr6YyTfQQ%2F640%3Fwx_fmt%3Dpng)

### Step2: 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

In [7]:
label = (data.source == '新华社').astype(int)
content = data.content

In [8]:
data_new = pd.DataFrame({'content':content, 'label':label})

In [13]:
data_new.head()

,content,label
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


In [191]:
len(data_new)

87054

### Step3: 使用tfidf进行文本向量化

#### 新闻内容切词整理 

In [10]:
ltp_model_path = "D:\\Github\\NLP\\Projects\\data\\ltp_data_v3.4.0"
segmentor_model_path = os.path.join(ltp_model_path, 'cws.model')
segmentor = Segmentor()
segmentor.load(segmentor_model_path)

In [11]:
def ltp_cut(document):
    result = ''
    if isinstance(document, str):
        words = segmentor.segment(document)
        words = [re.findall('\d*\w+', x) for x in words]
        result = ' '.join([x[0] for x in words if x and x[0] !='n'])  # pyltp会把换行符\n给分开
        return result if result else None

#### 切词保存数据，这一步还是要花几分钟的

In [16]:
data_new['content_cut'] = data_new.content.apply(lambda i: ltp_cut(i))
data_new.to_csv('cleaned_data.csv')

In [167]:
# index_col 参数不可省略，否则会出现未命名列
df = pd.read_csv('cleaned_data.csv', index_col=0)

In [173]:
# 删除 nan 无数据行，并重置索引为连续
df.dropna(inplace=True)

In [177]:
df.tail()

,content,label,content_cut
89606,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...
89607,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...
89608,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...
89609,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...
89610,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...


In [178]:
df.reset_index(drop=True, inplace=True)

In [179]:
df.tail()

,content,label,content_cut
86858,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...
86859,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...
86860,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...
86861,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...
86862,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...


#### 文本向量化

In [153]:
# 指定一定数量的feature，tfidf值小于前该数量的词汇，其tfidf值设置为 0
vectorized = TfidfVectorizer(max_features=10000)

In [154]:
X = vectorized.fit_transform(df.content_cut)

In [181]:
X.shape

(86863, 10000)

In [182]:
df['doc_vector'] = pd.DataFrame(X)

In [183]:
df.head()

,content,label,content_cut,doc_vector
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0,此外 自 本周 6月 12日 起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已...,"(0, 6335)\t0.11611960444487592\n (0, 3577)\..."
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0,骁龙 835 作为 唯一 通过 Windows 10 桌面 平台 认证 的 ARM 处理器 ...,"(0, 8886)\t0.07547001813218178\n (0, 6501)\..."
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0,此前 的 一 加 3T 搭载 的 是 3400mAh 电池 DashCharge 快充 规格...,"(0, 5775)\t0.22712698483179342\n (0, 4131)\..."
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1,这 是 6月 18日 在 葡萄牙 中部 大佩德罗冈 地区 拍摄 的 被 森林 大火 烧毁 的...,"(0, 8368)\t0.10868191347135386\n (0, 5286)\..."
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0,原 标题 44 岁 女子 跑 深圳 约会 网友 被 拒 暴雨 中 裸身 奔走 深圳交警 微博...,"(0, 8648)\t0.022683129804263208\n (0, 7489)..."


In [188]:
t1 = df.doc_vector[13].toarray()
t2 = df.doc_vector[2].toarray()

* scipy库中consine的实现为：$$cosine(u, v) = 1 - \frac{u \cdot v}{||u||_2 ||v||_2}$$  取值范围[0, 2];因此这个函数的结果越小说明越相似。

In [189]:
cosine(t1, t2)

0.9780645102825155

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. [reference](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

+ 参照scikit learning的示例，从构建你的第一个KNN机器学习模型。[reference](https://scikit-learn.org/stable/modules/neighbors.html#neighbors)

### Step5: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

### Step6: 模型测试test_data

### Step7: 调整不同的参数，观察变化

### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？

### Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

### (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。 

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。